In [1]:
import numpy as np
import pandas as pd
import os
import warnings

In [6]:
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

In [8]:
header = ['user_id','item_id','rating','timestamp']
dataset = pd.read_csv('../data/ml-100k/u.data',sep = '\t',names = header)
dataset.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [15]:
n_items = len(pd.unique(dataset['item_id']))
n_users = len(pd.unique(dataset['user_id']))
n_users, n_items

(943, 1682)

In [16]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [18]:
# creating book embedding path
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_items+1, 5, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Books")(movie_embedding)
# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)
# concatenate features
conc = Concatenate()([movie_vec, user_vec])
# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)
# Create model and compile it
model2 = Model([user_input, movie_input], out)
model2.compile('adam', 'mean_squared_error')
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Movie-Input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 User-Input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 Movie-Embedding (Embedding)    (None, 1, 5)         8415        ['Movie-Input[0][0]']            
                                                                                                  
 User-Embedding (Embedding)     (None, 1, 5)         4720        ['User-Input[0][0]']             
                                                                                            

In [19]:
history = model2.fit([train.user_id, train.item_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.1644
Epoch 2/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.8912
Epoch 3/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.8657
Epoch 4/5
2500/2500 [==============================] - 5s 2ms/step - loss: 0.8447
Epoch 5/5
2500/2500 [==============================] - 6s 2ms/step - loss: 0.8262


In [20]:
predictions = model2.predict([test.user_id.head(10), test.item_id.head(10)])
[print(predictions[i], test.rating.iloc[i]) for i in range(0,10)]

[3.3898656] 4
[3.7897916] 3
[3.167253] 4
[3.341616] 2
[3.8870332] 2
[3.0318882] 3
[3.7711072] 5
[3.91507] 4
[3.4157715] 3
[2.9789467] 4


[None, None, None, None, None, None, None, None, None, None]